In [4]:
# This notebook merges the curated dataset (only technical rock climbs) with the set of MP user ratings

import pandas as pd
import numpy as np
import glob

In [5]:
# my curated data, without rating data
nr_df = pd.read_pickle('Curated_OpenBetaAug2020_RytherAnderson.pkl.zip')
nr_df['route_ID'] = nr_df['route_ID'].astype(int)

In [6]:
# all the rating files are concatenated

rating_files = glob.glob('../ratings/*.zip')
AR = pd.read_csv(rating_files[0], compression='zip')

for rf in rating_files[1:]:
    df = pd.read_csv(rf, compression='zip')
    join = [AR, df]
    AR = pd.concat(join)
    
AR.to_pickle('All_Ratings.pkl.zip', compression='zip')
AR = pd.read_pickle('All_Ratings.pkl.zip', compression='zip')
unique_ids = set(AR.route_id)
print(len(unique_ids), 'routes with star ratings')

99682 routes with star ratings


In [7]:
# this cell takes a long time, evaluate only if you are ready, probably I wrote it in a silly way
# here the list of star ratings and users are added to each route

ratings_df = nr_df[nr_df['route_ID'].isin(unique_ids)].copy()
ratings_df['ratings'] = [list(AR[AR.route_id == id]['ratings']) for id in ratings_df.route_ID]
ratings_df['users'] = [list(AR[AR.route_id == id]['users']) for id in ratings_df.route_ID]

ratings_df.head()
ratings_df.to_pickle('CuratedWithRatings_OpenBetaAug2020_RytherAnderson.pkl')